In [2]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# Verify MEPS Graph

In [ ]:
import torch
import os

graph_dir_path = "/vol/bitbucket/bet20/neural-lam/graphs/multiscale"
static_dir_path = "/vol/bitbucket/bet20/neural-lam/data/meps_example/static"

In [ ]:
grid_features = torch.load(os.path.join(static_dir_path, "grid_features.pt"))
print(grid_features.shape)

mesh_features = torch.load(os.path.join(graph_dir_path, "mesh_features.pt"))
print(mesh_features[0].shape)

torch.Size([63784, 4])
torch.Size([6561, 2])


In [ ]:
g2m_features = torch.load(os.path.join(graph_dir_path, "g2m_features.pt"))
g2m_edge_index = torch.load(os.path.join(graph_dir_path, "g2m_edge_index.pt"))

print(g2m_edge_index.shape)

gmin, gmax = g2m_edge_index[0].min(), g2m_edge_index[0].max()
mmin, mmax = g2m_edge_index[1].min(), g2m_edge_index[1].max()
print(gmin, gmax, mmin, mmax)
print(gmax - gmin + 1, mmax - mmin + 1)
print(g2m_features.shape[0])

# print(g2m_edge_index.min(dim=1, keepdim=True)[0])
# print(g2m_edge_index.min(dim=1, keepdim=True).values[0])

g2m_edge_index = g2m_edge_index - g2m_edge_index.min(dim=1, keepdim=True)[0]

torch.Size([2, 100656])
tensor(6561) tensor(70344) tensor(0) tensor(6560)
tensor(63784) tensor(6561)
100656


In [ ]:
g2m_edge_index

tensor([[    0,     1,     2,  ..., 63781, 63782, 63783],
        [    0,     0,     0,  ...,  6560,  6560,  6560]])

In [ ]:
num_rec = g2m_edge_index[1].max() + 1

In [ ]:
g2m_edge_index[0] = (
    g2m_edge_index[0] + num_rec
)

print(g2m_edge_index)

tensor([[    0,     1,     2,  ..., 63781, 63782, 63783],
        [-6561, -6561, -6561,  ...,    -1,    -1,    -1]])


# Sanity Check Graph

In [3]:
import numpy as np
import torch
import os

In [4]:
dir = "./data/era5_uk/static/nwp_xy.npy"
grid_xy = torch.tensor(np.load(dir))
_, lon, lat = grid_xy.shape

graph_name = "uk_graphcast"
graph_dir_path = os.path.join("graphs", graph_name)

In [ ]:
print(lon, lat)
print("Local grid nodes", lon * lat)

X, Y = 1440, 721
fraction = (lon * lat) / (X * Y)
print("Global grid nodes", X * Y)
print("Local : Global", fraction)

57 65
Local grid nodes 3705
Global grid nodes 1038240
Local : Global 0.0035685390661118815


### Verify M2M Graph

In [ ]:
N_MESH_NODES = 40962
N_MESH_EDGES = 327660 

mesh_features = torch.load(os.path.join(graph_dir_path, "mesh_features.pt"))
m2m_features = torch.load(os.path.join(graph_dir_path, "m2m_features.pt"))
m2m_edge_index = torch.load(os.path.join(graph_dir_path, "m2m_edge_index.pt"))

print(mesh_features[0].shape)
print(m2m_features[0].shape)
print(m2m_edge_index[0].shape)
print(m2m_edge_index[0].unique().shape)

print("Expected mesh nodes", N_MESH_NODES * fraction)
print("Expected mesh edges", N_MESH_EDGES * fraction)

torch.Size([355, 3])
torch.Size([2584, 4])
torch.Size([2, 2584])
torch.Size([355])
Expected mesh nodes 146.1744972260749
Expected mesh edges 1169.2675104022192


In [ ]:
N_G2M_EDGES = 1618746

g2m_features = torch.load(os.path.join(graph_dir_path, "g2m_features.pt"))
g2m_edge_index = torch.load(os.path.join(graph_dir_path, "g2m_edge_index.pt"))

print(g2m_features.shape)
print(g2m_edge_index.shape)
print(g2m_edge_index[0].unique().shape)
print(g2m_edge_index[1].unique().shape)
print(lat * lon)
print("Expected g2m edges", N_G2M_EDGES * fraction)

torch.Size([7719, 4])
torch.Size([2, 7719])
torch.Size([3705])
torch.Size([355])
3705
Expected g2m edges 5776.558339112344


### Verify G2M Graph

### Verify M2G Graph

In [ ]:
N_M2G_EDGES = 3114720

m2g_features = torch.load(os.path.join(graph_dir_path, "m2g_features.pt"))
m2g_edge_index = torch.load(os.path.join(graph_dir_path, "m2g_edge_index.pt"))

print(m2g_features.shape)
print(m2g_edge_index.shape)
print(m2g_edge_index[0].unique().shape)
print(m2g_edge_index[1].unique().shape)
print(lat * lon)
print("Expected m2g edges", N_M2G_EDGES * fraction)

torch.Size([11094, 4])
torch.Size([2, 11094])
torch.Size([355])
torch.Size([3705])
3705
Expected m2g edges 11115.0


### Verify against full graphcast

In [ ]:
# num_grid_nodes = 
num_m2m_edges = 327660
num_m2g_edges = 3114720
  

```
g2m graph: Graph(num_nodes={'grid': 3705, 'mesh': 355},
      num_edges={('grid', 'g2m', 'mesh'): 7719},
      metagraph=[('grid', 'mesh', 'g2m')])
m2g graph: Graph(num_nodes={'grid': 3705, 'mesh': 355},
      num_edges={('mesh', 'm2g', 'grid'): 11094},
      metagraph=[('mesh', 'grid', 'm2g')])
mesh graph: Graph(num_nodes=355, num_edges=2584,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(4,), dtype=torch.float32)})
```